In [ ]:
# intall
!pip install telebot
!pip install pyTelegramBotAPI
!pip install telegram
!pip install python-telegram-bot
!pip install telethon

In [ ]:
# import
import os
import telebot
import requests
import time
import telegram

import shutil
import time
import pandas as pd
from telebot import types
from threading import Thread

In [ ]:
# TelegramBot_Final
BOT_TOKEN = 'here_comes_your_code_iHuuCVrkPthBJwo2gA95M-suB_LDffKs'

def run_bot():
    while True:
        try:
            bot = telebot.TeleBot(BOT_TOKEN)
            users_sent_autoreply = {}

            @bot.message_handler(commands=['start'])
            def send_welcome(message):
                welcome_message = "Hello there! 👋\n\n" \
                                 "Welcome to the _YOUR_COMPANY_NAME_, where you can get rewards by uploading your _KIND_OF_DATA_YOU_NEED_ to our AI training data directory 🤖\n" \
                                 "Please take screenshots or screen capture videos of your _KIND_OF_DATA_YOU_NEED_, and send them here 💬"

                bot.send_message(message.chat.id, welcome_message)

            @bot.message_handler(content_types=['photo', 'video'])
            def handle_uploaded_file(message):
                file_id = message.photo[-1].file_id if message.content_type == 'photo' else message.video.file_id
                file_info = bot.get_file(file_id)
                file_path = file_info.file_path
                file_url = "https://api.telegram.org/file/bot{}/{}".format(BOT_TOKEN, file_path)
                response = requests.get(file_url)

                user_id = message.from_user.id
                file_extension = file_path.split('.')[-1]
                new_file_name = f"{file_path.replace('.', f'{user_id}{message.text}.')}"

                save_path = os.path.join('/content/sample_data', new_file_name)
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                with open(save_path, 'wb') as file:
                    file.write(response.content)

                if user_id not in users_sent_autoreply:
                    keyboard = types.InlineKeyboardMarkup()
                    website_button = types.InlineKeyboardButton(text='Website', url='https://YOURWEBSITE/')
                    whitepaper_button = types.InlineKeyboardButton(text='Whitepaper', url='https://YOURWEBSITE.documentation.suave.chat/')
                    telegram_button = types.InlineKeyboardButton(text='Telegram', url='https://t.me/YOURWEBSITE')
                    keyboard.add(website_button, whitepaper_button, telegram_button)

                    bot.send_message(message.chat.id, "Thank you for your contribution! We've received it successfully and will begin processing it shortly ⌛️\n\n" \
                                      "Before you leave, please provide your wallet address so we can distribute your reward once it is calculated 💰")

                    users_sent_autoreply[user_id] = True

            @bot.message_handler(func=lambda message: len(message.text) == 48)
            def handle_id(message):
                bot.send_message(message.chat.id, "Your training data is now under processing. This may take a few days, so please be patient ⌛")
                time.sleep(10)

                keyboard = types.InlineKeyboardMarkup()
                website_button = types.InlineKeyboardButton(text='Website', url='https://YOURWEBSITE/')
                whitepaper_button = types.InlineKeyboardButton(text='Whitepaper', url='https://YOURWEBSITE.documentation.suave.chat/')
                telegram_button = types.InlineKeyboardButton(text='Telegram', url='https://t.me/YOURWEBSITE')
                keyboard.add(website_button, whitepaper_button, telegram_button)

                final_message = "While you're waiting for your training data to be evaluated, you might find our website and whitepaper helpful and don’t forget to also join our Telegram group for more updates."
                bot.send_message(message.chat.id, final_message, reply_markup=keyboard)

            @bot.message_handler(func=lambda message: len(message.text) != 48)
            def handle_invalid_id(message):
                bot.send_message(message.chat.id, "Ooops, this looks like an invalid TON address... 😱 Please check your TON wallet and send a valid TON address.")

            bot.polling()
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            time.sleep(60)

bot_thread = Thread(target=run_bot)
bot_thread.start()

while True:
    time.sleep(60 * 60)  # Wait for 1 hour
    bot_thread.join()
    bot_thread = Thread(target=run_bot)
    bot_thread.start()
